<a href="https://colab.research.google.com/github/keunsooyoon/crawling/blob/main/%EC%8A%A4%ED%83%80%EB%B2%85%EC%8A%A4_%EC%BD%94%EB%9E%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 스타벅스 입지 분석

# 코랩에서 chromedriver 사용하는 문법

!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver/usr/bin

from selenium import webdriver

options =webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)



     |████████████████████████████████| 968 kB 5.6 MB/s 
     |████████████████████████████████| 138 kB 48.0 MB/s 
     |████████████████████████████████| 359 kB 42.0 MB/s 
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 3.6 MB 45.4 MB/s 
     |████████████████████████████████| 58 kB 4.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/

In [54]:

url = 'https://www.starbucks.co.kr/store/store_map.do'
driver.get(url)


In [55]:
btn_area = '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
driver.find_element_by_css_selector(btn_area).click()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  


In [56]:
btn_seoul = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(btn_seoul).click()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  


In [57]:
btn_all = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(btn_all).click()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  


In [58]:
html = driver.page_source


In [59]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html)

In [60]:
len(soup.select('li.quickResultLstCon'))

584

In [61]:
starbucks_list = soup.select('li.quickResultLstCon')

In [62]:
print(starbucks_list[0])

<li class="quickResultLstCon" data-code="3969" data-hlytag="null" data-index="0" data-lat="37.567399" data-long="126.978900" data-name="무교로" data-storecd="1724" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="무교로" data-store="1724" data-yn="N">무교로  </strong> <p class="result_details">서울특별시 중구 무교로 15 (무교동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>


In [63]:
starbucks = starbucks_list[0]

#매장명
name = starbucks.select('strong')[0].text.strip()

#위도
lat = starbucks['data-lat']

#경도
lng = starbucks['data-long']

#주소
addr = str(starbucks.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]

#전화번호
tel = str(starbucks.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]


In [65]:
import time

starbucks_seoul_list = []
for star in starbucks_list:
  #매장명
  name = star.select('strong')[0].text.strip()
  #위도
  lat = star['data-lat']
  #경도
  lng = star['data-long']
  #주소
  addr = str(star.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
  #전화번호
  tel = str(star.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

  time.sleep(0.01)
  starbucks_seoul_list.append([name, lat, lng, addr, tel])



In [67]:
import pandas as pd
df = pd.DataFrame(starbucks_seoul_list, columns = ['매장명','위도','경도','주소','전화번호'])

df

,매장명,위도,경도,주소,전화번호
0,무교로,37.567399,126.978900,서울특별시 중구 무교로 15 (무교동),1522-3232
1,한국프레스센터,37.567381,126.977859,서울특별시 중구 세종대로 124 (태평로1가),1522-3232
2,무교동,37.56781712,126.978738,서울특별시 중구 무교로 21 (무교동) 코오롱빌딩 1층,1522-3232
3,시청,37.566290,126.979808,"서울특별시 중구 을지로 19, 삼성화재삼성빌딩 1층 (을지로1가)",1522-3232
4,환구단,37.564448,126.979157,서울특별시 중구 소공로 112 (소공동),1522-3232
...,...,...,...,...,...
579,중랑구청,37.60538908,127.0957558,서울특별시 중랑구 신내로 72,1522-3232
580,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232
581,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
582,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232


In [68]:
df.to_excel('starbucks_seoul_list.xlsx', index = False)